In [1]:
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
import sys

In [2]:
def ingest():
    # Get the doc
    loader = PyPDFLoader("Employee-Handbook.pdf")
    pages = loader.load_and_split()
    # Split the pages by char
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1024,
        chunk_overlap=100,
        length_function=len,
        add_start_index=True,
    )
    chunks = text_splitter.split_documents(pages)
    print(f"Split {len(pages)} documents into {len(chunks)} chunks.")
    #
    embedding = FastEmbedEmbeddings()
    #Create vector store
    Chroma.from_documents(documents=chunks,  embedding=embedding, persist_directory="./emp_chr_db")

In [7]:
ingest()

Split 34 documents into 75 chunks.


In [6]:



from huggingface_hub import login
access_token_read =""
access_token_write = ""
login(token = access_token_read)



### Create a RAG chain that retreives relevent chunks and prepares a response

In [8]:
def rag_chain():
    model = ChatOllama(model="gemma3:4b")
    #
    prompt = PromptTemplate.from_template(
        """
        <s> [Instructions] You are a friendly assistant. Answer the question based only on the following context. 
        If you don't know the answer, then reply, No Context availabel for this question {input}. [/Instructions] </s> 
        [Instructions] Question: {input} 
        Context: {context} 
        Answer: [/Instructions]
        """
    )
    #Load vector store
    embedding = FastEmbedEmbeddings()
    vector_store = Chroma(persist_directory="./emp_chr_db", embedding_function=embedding)

    #Create chain
    retriever = vector_store.as_retriever(
        search_type="similarity_score_threshold",
        search_kwargs={
            "k": 3,
            "score_threshold": 0.5,
        },
    )

    document_chain = create_stuff_documents_chain(model, prompt)
    chain = create_retrieval_chain(retriever, document_chain)
    #
    return chain

In [9]:
def ask(query: str):
    #
    chain = rag_chain()
    # invoke chain
    result = chain.invoke({"input": query})
    # print results
    print(result["answer"])
    for doc in result["context"]:
        print("Source: ", doc.metadata["source"])

### Ask Question to get relevant information from the document

In [10]:
ask("What are steps involved in the Recruitment and selection process")

C:\Users\nisam\AppData\Local\Temp\ipykernel_15364\2256481245.py:2: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  model = ChatOllama(model="gemma3:4b")
C:\Users\nisam\AppData\Local\Temp\ipykernel_15364\2256481245.py:15: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_store = Chroma(persist_directory="./sql_chroma_db", embedding_function=embedding)


Here are the steps involved in the recruitment and selection process:

i. Identify the need for a new job opening.
ii. Decide whether to hire externally or internally.
iii. Review job descriptions and write a job ad.
iv. Get approval for your job ad.
v. Select appropriate sources (external or internal) to post your job opening.
vi. Decide on hiring stages and possible timeframes.
vii. Review resumes in our company database/ATS.
viii. Source passive candidates.
ix. Shortlist applicants.
Source:  Employee-Handbook.pdf
Source:  Employee-Handbook.pdf
Source:  Employee-Handbook.pdf


In [15]:
ask("What are all they provding for safety and health ")

They are providing for safety and health through:

*   Preventative action, including periodic risk assessments and job hazard analyses conducted by a workplace safety committee.
*   Establishing preventative measures to address risks.
Source:  Employee-Handbook.pdf
Source:  Employee-Handbook.pdf
Source:  Employee-Handbook.pdf


In [16]:
ask("What are steps involved in Recruitment and selection process ")

Here are the steps involved in the recruitment and selection process:

i. Identify the need for a new job opening.
ii. Decide whether to hire externally or internally.
iii. Review job descriptions and write a job ad.
iv. Get approval for your job ad.
v. Select appropriate sources (external or internal) to post your job opening.
vi. Decide on hiring stages and possible timeframes.
vii. Review resumes in our company database/ATS.
viii. Source passive candidates.
ix. Shortlist applicants.
Source:  Employee-Handbook.pdf
Source:  Employee-Handbook.pdf
Source:  Employee-Handbook.pdf
